# Course Recommendation System

In [4]:
# import libraries
import pandas as pd
import numpy as np
import neattext.functions as nfx
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
data = pd.read_csv("dataset.csv")
data.head()

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,profit,published_date,published_time,year,month,day
0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...,True,200,2147,23,51,All Levels,1.5 hours,2017-01-18T20:58:58Z,Business Finance,429400,2017-01-18,20:58:58Z,2017,1,18
1,1113822,Complete GST Course & Certification - Grow You...,https://www.udemy.com/goods-and-services-tax/,True,75,2792,923,274,All Levels,39 hours,2017-03-09T16:34:20Z,Business Finance,209400,2017-03-09,16:34:20Z,2017,3,9
2,1006314,Financial Modeling for Business Analysts and C...,https://www.udemy.com/financial-modeling-for-b...,True,45,2174,74,51,Intermediate Level,2.5 hours,2016-12-19T19:26:30Z,Business Finance,97830,2016-12-19,19:26:30Z,2016,12,19
3,1210588,Beginner to Pro - Financial Analysis in Excel ...,https://www.udemy.com/complete-excel-finance-c...,True,95,2451,11,36,All Levels,3 hours,2017-05-30T20:07:24Z,Business Finance,232845,2017-05-30,20:07:24Z,2017,5,30
4,1011058,How To Maximize Your Profits Trading Options,https://www.udemy.com/how-to-maximize-your-pro...,True,200,1276,45,26,Intermediate Level,2 hours,2016-12-13T14:57:18Z,Business Finance,255200,2016-12-13,14:57:18Z,2016,12,13


In [6]:
data["Clean_Title"] = data["course_title"].apply(nfx.remove_stopwords)
data["Clean_Title"] = data["Clean_Title"].apply(nfx.remove_special_characters)

data.head()

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,profit,published_date,published_time,year,month,day,Clean_Title
0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-bank...,True,200,2147,23,51,All Levels,1.5 hours,2017-01-18T20:58:58Z,Business Finance,429400,2017-01-18,20:58:58Z,2017,1,18,Ultimate Investment Banking Course
1,1113822,Complete GST Course & Certification - Grow You...,https://www.udemy.com/goods-and-services-tax/,True,75,2792,923,274,All Levels,39 hours,2017-03-09T16:34:20Z,Business Finance,209400,2017-03-09,16:34:20Z,2017,3,9,Complete GST Course Certification Grow Practice
2,1006314,Financial Modeling for Business Analysts and C...,https://www.udemy.com/financial-modeling-for-b...,True,45,2174,74,51,Intermediate Level,2.5 hours,2016-12-19T19:26:30Z,Business Finance,97830,2016-12-19,19:26:30Z,2016,12,19,Financial Modeling Business Analysts Consultants
3,1210588,Beginner to Pro - Financial Analysis in Excel ...,https://www.udemy.com/complete-excel-finance-c...,True,95,2451,11,36,All Levels,3 hours,2017-05-30T20:07:24Z,Business Finance,232845,2017-05-30,20:07:24Z,2017,5,30,Beginner Pro Financial Analysis Excel 2017
4,1011058,How To Maximize Your Profits Trading Options,https://www.udemy.com/how-to-maximize-your-pro...,True,200,1276,45,26,Intermediate Level,2 hours,2016-12-13T14:57:18Z,Business Finance,255200,2016-12-13,14:57:18Z,2016,12,13,Maximize Profits Trading Options


In [7]:
# Vectorise the "Clean Title" column
countvect = CountVectorizer()
cvmat = countvect.fit_transform(data["Clean_Title"])
cvmat

# 3654 features have been generated 

<3683x3564 sparse matrix of type '<class 'numpy.int64'>'
	with 18364 stored elements in Compressed Sparse Row format>

In [9]:
# cosine similarity 

cos_sim = cosine_similarity(cvmat)
cos_sim

array([[1.        , 0.20412415, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.20412415, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.23570226],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.23570226, 0.        ,
        1.        ]])

In [10]:
cos_sim.shape

(3683, 3683)

In [14]:
# Recommend Course
course_index = pd.Series(data.index, index=data["course_title"]).drop_duplicates()

index = course_index[title]

scores = list(enumerate(cos_sim[index]))

sorted_scores = sorted(scores, key = lambda x: x[1], reverse=True)

selected_course_index = [i[0] for i in sorted_scores[1:]]
selected_course_score = [i[1] for i in sorted_scores[1:]]

rec_df = data.iloc[selected_course_index]

rec_df["Similarity_Score"] = selected_course_score

final_recommended_courses = rec_df[["course_title", "Similarity_Score", "url", "price", "num_subscribers"]]

final_recommended_courses

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,course_title,Similarity_Score,url,price,num_subscribers
410,Trading Options Basics,0.577350,https://www.udemy.com/trading-options-basics/,200,8
43,Options Trading - How to Win with Weekly Options,0.566947,https://www.udemy.com/work-from-home-setup-you...,115,7489
96,Intermediate Options trading concepts for Stoc...,0.530330,https://www.udemy.com/intermediate-options-tra...,40,2000
138,Forex Trading with Fixed 'Risk through Options...,0.530330,https://www.udemy.com/forexoptions/,200,611
195,Trading Options For Consistent Returns: Option...,0.530330,https://www.udemy.com/trading-options-for-income/,0,4077
...,...,...,...,...,...
3678,Learn jQuery from Scratch - Master of JavaScri...,0.000000,https://www.udemy.com/easy-jquery-for-beginner...,100,1040
3679,How To Design A WordPress Website With No Codi...,0.000000,https://www.udemy.com/how-to-make-a-wordpress-...,25,306
3680,Learn and Build using Polymer,0.000000,https://www.udemy.com/learn-and-build-using-po...,40,513
3681,CSS Animations: Create Amazing Effects on Your...,0.000000,https://www.udemy.com/css-animations-create-am...,50,300
